# ML Powered Applications

# Chapter 4

Data preration notebook to better understand the data.

Objective
- To produce an etl capable of producing datasets on request

Process
- Import data
- Clean data types
- Flag outliers

In [1]:
# library dependencies
import pandas as pd
import numpy as np
import re
from datetime import datetime

# intialization
PATH_data = r"C:\Users\nrosh\Desktop\Personal Coding Projects\Python\ml-powered-applications\neel\data"

## Domain research

Data structure
1. PostType: 
    - {1: Question, 2: Answer}
2. AcceptedAnswerId: 
    - Row Id that answers a id. One Id can have many AcceptedAnswerId's
3. Score: 
    - Number of upvotes - Number of downvotes
4. ViewCount: 
    - Number of post views
5. body_text: 
    - Text associate with row entry id
6. FavoriteCount: 
    - Number of times the post was favorited
7. CommentCount: 
    - Number of times a comment was made on the Post
8. CreationDate: 
    - Date the post was created
9. LastActivityDate:
    - datetime of the post's most recent activity
10. LastEditDate
    - The date and time of the most recent edit to the post
11. ParentId
    - Row whose Parent is ParentId
Sources:
- stackexchange: 
    - Data Schema: https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede
    - Score: https://meta.stackexchange.com/questions/229255/what-is-the-score-of-a-post
    

## Functions

In [2]:
# Convert a value a datatype if not null
def convert_to(val, typefunc):
    if not np.isnan(val):
        return typefunc(val)
    else:
        return val

## Ingestion


In [3]:
# original
df_orig = pd.read_csv(
    PATH_data + "\\raw\Posts.csv", 
    index_col='Id'
)

# copies
df = df_orig.copy()

## Data Cleansing

In [4]:
# Define required columns
columns = [
    "PostTypeId", "AcceptedAnswerId", "ParentId", "AnswerCount", 
    "CommentCount", "FavoriteCount", "LastActivityDate", "CreationDate",
    "ClosedDate", "LastEditDate", "Score", "Title", "body_text" 
          ]

df1 = df.copy()
df1 = df1.loc[:, columns]

In [5]:
# id type conversion
int_cols = ["AcceptedAnswerId", "ParentId"]
date_cols = ["LastActivityDate", "CreationDate", "ClosedDate", "LastEditDate"]

# convert to integer
for col in int_cols:
    df1.loc[:, col] = df1.loc[:, col].apply(lambda row: convert_to(row, int))

# convert to datetime
for col in date_cols:
    df1.loc[:, col] = pd.to_datetime(df1.loc[:, col])
    
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42605 entries, 1 to 52454
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   PostTypeId        42605 non-null  int64         
 1   AcceptedAnswerId  5081 non-null   float64       
 2   ParentId          31935 non-null  float64       
 3   AnswerCount       9935 non-null   float64       
 4   CommentCount      42605 non-null  int64         
 5   FavoriteCount     4052 non-null   float64       
 6   LastActivityDate  42605 non-null  datetime64[ns]
 7   CreationDate      42605 non-null  datetime64[ns]
 8   ClosedDate        1294 non-null   datetime64[ns]
 9   LastEditDate      15252 non-null  datetime64[ns]
 10  Score             42605 non-null  int64         
 11  Title             9935 non-null   object        
 12  body_text         42519 non-null  object        
dtypes: datetime64[ns](4), float64(4), int64(3), object(2)
memory usage: 4.6+ MB


## Feature Engineering

1. Closed question -> Where ClosedDate is NaT
2. question/answer -> Differentiate between a question, answer and everything else
3. Comment Count -> High, medium, low based on histogram
4. Score Count -> High, medium, low based on histogram

In [6]:
# function definitions

In [7]:
# make copy
df2 = df1.copy()

In [8]:
# define new features here
df2["fe_tenure"]         = np.inf # time to close questions in hours
df2["fe_isclosed"]       = 0 # open vs closed question
df2["fe_isquestion"]     = 0 # question or answer
df2["fe_isanswer"]       = 0 # question or answer
df2["fe_isfavorited"]    = 0 # whether post was favorited or not
df2["fe_wasedited"]      = 0 # whether post was edited or not
df2["fe_question_answered"] = 0 # whehter a post was answered or not

In [9]:
# feature engineer - question/answer
df2.loc[df2["PostTypeId"] == 1, "fe_isquestion"]      = df2.loc[df2["PostTypeId"] == 1, "fe_isquestion"].apply(lambda row: 1)
df2.loc[df2["PostTypeId"] == 2, "fe_isanswer"]        = df2.loc[df2["PostTypeId"] == 2, "fe_isanswer"].apply(lambda row: 1)
df2.loc[df2["FavoriteCount"] > 0, "fe_isfavorited"]   = df2.loc[df2["FavoriteCount"] > 0, "fe_isfavorited"].apply(lambda row: 1)
df2.loc[df2.LastEditDate.isna(), "fe_wasedited"]      = df2.loc[df2.LastEditDate.isna(), "fe_wasedited"].apply(lambda row: 1)

# feature engineer - tenure of post in hours
df2.loc[:, 'fe_tenure'] = (df2["ClosedDate"] - df2["CreationDate"]).apply(
    lambda row: np.inf \
        if pd.isnull(row) \
        else (row.days)*24   
)
df2.head()

# feature engineer - post is open/closed
df2.loc[:, "fe_isclosed"] = df2["fe_tenure"].apply(lambda row: 0 if row == np.inf else 1)

# feature engineer whether a question was answered or not
df2.loc[(df2["AcceptedAnswerId"]) > 0, "fe_question_answered"] = 1

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42605 entries, 1 to 52454
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   PostTypeId            42605 non-null  int64         
 1   AcceptedAnswerId      5081 non-null   float64       
 2   ParentId              31935 non-null  float64       
 3   AnswerCount           9935 non-null   float64       
 4   CommentCount          42605 non-null  int64         
 5   FavoriteCount         4052 non-null   float64       
 6   LastActivityDate      42605 non-null  datetime64[ns]
 7   CreationDate          42605 non-null  datetime64[ns]
 8   ClosedDate            1294 non-null   datetime64[ns]
 9   LastEditDate          15252 non-null  datetime64[ns]
 10  Score                 42605 non-null  int64         
 11  Title                 9935 non-null   object        
 12  body_text             42519 non-null  object        
 13  fe_tenure       

## Data Merging

- Produce the following dataframe;
1. questions_asked
2. answer_provided

SQL

    SELECT 
        df1.Title as Title,
        df1.body_text as question
        df2.body_text as accepted_answer
    FROM df2 as d1
    LEFT INNER JOIN df2 as d2
    ON df1.AcceptedAnswerId = df2.Id

In [10]:
df3 = df2.copy()
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42605 entries, 1 to 52454
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   PostTypeId            42605 non-null  int64         
 1   AcceptedAnswerId      5081 non-null   float64       
 2   ParentId              31935 non-null  float64       
 3   AnswerCount           9935 non-null   float64       
 4   CommentCount          42605 non-null  int64         
 5   FavoriteCount         4052 non-null   float64       
 6   LastActivityDate      42605 non-null  datetime64[ns]
 7   CreationDate          42605 non-null  datetime64[ns]
 8   ClosedDate            1294 non-null   datetime64[ns]
 9   LastEditDate          15252 non-null  datetime64[ns]
 10  Score                 42605 non-null  int64         
 11  Title                 9935 non-null   object        
 12  body_text             42519 non-null  object        
 13  fe_tenure       

In [19]:
def export_df(df, cols, **kwargs):

    # rename and return a dataframe of those columns
    # choose to export or not
    
    _df = df.loc[:, cols]
    
    # rename dict
    if "rename_dict" in kwargs.keys() :
        _df.rename(columns=kwargs["rename_dict"], inplace=True)
        print('- Columns renamed.')
    
    # export data
    if kwargs["export_loc"]:
        
        # handle data export
        try:
            
            if "export_name" in kwargs.keys():
                _location = kwargs["export_loc"] + "\\{}".format(kwargs["export_name"]) + ".csv"
                _df.to_csv(_location)
                print(f"""- File exported to: {_location}""")
            else:
                _location = kwargs["export_loc"]+"\\adhoc_{}".format(datetime.today().strftime("%m%d%y"))+".csv"
                _df.to_csv(_location)
                print(f"""- File exported to: {_location}""")

        except:
            raise Exception(f"""export_loc must be of type str. Given: {type(kwargs["export_loc"])}""")
    
    print('\n')
    return _df

In [12]:
# prepare dataframes
df_questions = df3.loc[df3["fe_isquestion"] == 1, :]
df_answers   = df3.loc[df3["fe_isanswer"] == 1, :]

# reset indexes
df_questions.reset_index(inplace=True)
df_answers.reset_index(inplace=True)

df_questions.to_csv(PATH_data+"\\out\post_questions.csv")
df_answers.to_csv(PATH_data+"\\out\post_answers.csv")

In [13]:
# left inner join questions and answers
df_qa = df_questions\
    .merge(df_answers, left_on="AcceptedAnswerId", right_on="Id", how="inner")

df_qa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5081 entries, 0 to 5080
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Id_x                    5081 non-null   int64         
 1   PostTypeId_x            5081 non-null   int64         
 2   AcceptedAnswerId_x      5081 non-null   float64       
 3   ParentId_x              0 non-null      float64       
 4   AnswerCount_x           5081 non-null   float64       
 5   CommentCount_x          5081 non-null   int64         
 6   FavoriteCount_x         2399 non-null   float64       
 7   LastActivityDate_x      5081 non-null   datetime64[ns]
 8   CreationDate_x          5081 non-null   datetime64[ns]
 9   ClosedDate_x            398 non-null    datetime64[ns]
 10  LastEditDate_x          3254 non-null   datetime64[ns]
 11  Score_x                 5081 non-null   int64         
 12  Title_x                 5081 non-null   object  

In [14]:
# map questions with answered

# columns to select
cols = [
    "Title_x", "body_text_x", "body_text_y", 
    "CommentCount_x", "AnswerCount_x", "FavoriteCount_x", 
    "CommentCount_y", "Score_x", "fe_tenure_x", 
    "LastEditDate_x", "LastEditDate_y",
    "LastActivityDate_x", "LastActivityDate_y",
    "fe_isclosed_x", "fe_wasedited_x", "fe_wasedited_y"
]

# columns to rename
rename_dict = {
    "Title_x"             : "post_title", 
    "body_text_x"         : "post_question", 
    "body_text_y"         : "post_acceptedanswer",
    "Score_x"             : "post_score",
    "CommentCount_x"      : "postcomments_count",
    "AnswerCount_x"       : "postanswers_count",
    "FavoriteCount_x"     : "postfavorites_count",
    "CommentCount_y"      : "answercomments_count",
    "fe_tenure_x"         : "tenure_hours",
    "fe_isclosed_x"       : "post_closed",
    "fe_wasedited_x"      : "question_edited", 
    "fe_wasedited_y"      : "answer_edited",
    "LastActivityDate_x"  : "question_lastactivity",
    "LastActivityDate_y"  : "answer_lastactivity",
    "LastEditDate_x"      : "question_lastedited",
    "LastEditDate_y"      : "answer_lastedited"
}

In [15]:
# mapped questions with answers
export_loc = PATH_data+"\\out"

df4 = export_df(df_qa, cols, rename_dict=rename_dict, export_loc=export_loc)
df4.info()

- Columns renamed.
- File exported to: C:\Users\nrosh\Desktop\Personal Coding Projects\Python\ml-powered-applications\neel\data\out\adhoc_012421.csv


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5081 entries, 0 to 5080
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   post_title             5081 non-null   object        
 1   post_question          5081 non-null   object        
 2   post_acceptedanswer    5081 non-null   object        
 3   postcomments_count     5081 non-null   int64         
 4   postanswers_count      5081 non-null   float64       
 5   postfavorites_count    2399 non-null   float64       
 6   answercomments_count   5081 non-null   int64         
 7   post_score             5081 non-null   int64         
 8   tenure_hours           5081 non-null   float64       
 9   question_lastedited    3254 non-null   datetime64[ns]
 10  answer_lastedited      1756 n

In [20]:
# special requests

# columns to select
cols_s = [
    'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'AnswerCount',
    'CommentCount', 'FavoriteCount', 'LastActivityDate', 'CreationDate',
    'ClosedDate', 'LastEditDate', 'Score', 'Title', 'body_text',
    'fe_tenure', 'fe_isclosed', 'fe_isquestion', 'fe_isanswer',
    'fe_isfavorited', 'fe_wasedited', 'fe_question_answered'
]


In [23]:
# special requests
export_loc = PATH_data+"\\out"

df_special = export_df(df3, cols_s, export_loc=export_loc, export_name="cleaned_data")
df_special.info()

- File exported to: C:\Users\nrosh\Desktop\Personal Coding Projects\Python\ml-powered-applications\neel\data\out\cleaned_data.csv


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42605 entries, 1 to 52454
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   PostTypeId            42605 non-null  int64         
 1   AcceptedAnswerId      5081 non-null   float64       
 2   ParentId              31935 non-null  float64       
 3   AnswerCount           9935 non-null   float64       
 4   CommentCount          42605 non-null  int64         
 5   FavoriteCount         4052 non-null   float64       
 6   LastActivityDate      42605 non-null  datetime64[ns]
 7   CreationDate          42605 non-null  datetime64[ns]
 8   ClosedDate            1294 non-null   datetime64[ns]
 9   LastEditDate          15252 non-null  datetime64[ns]
 10  Score                 42605 non-null  int64         
 11  

## Sanity check's

In [24]:
# get questions received no answer
# how many received at least one
# how many had an answer that was accepted

df_hasnoanswer       = ((df3.fe_isquestion == 1) & (df3.AcceptedAnswerId.isna()) & (df3.AnswerCount < 1))
df_morethanoneanswer = ((df3.loc[df3.fe_isanswer == 1, "ParentId"].value_counts() > 1) == True)
df_acceptedAnswers   = df3.loc[(df3.fe_isquestion == 1) & (df3.AcceptedAnswerId.isna() == False)]

In [25]:
print(f"""

Questions that,

1. Recieved no answer: {df_hasnoanswer.value_counts().loc[True]}
2. Recieved more than one answer: {df_morethanoneanswer.value_counts().loc[True]} 
3. Recieved an answer that was accepted: {len(df_acceptedAnswers.index)}
""")



Questions that,

1. Recieved no answer: 257
2. Recieved more than one answer: 7234 
3. Recieved an answer that was accepted: 5081

